<a href="https://colab.research.google.com/github/delomeo/fairness-ml/blob/main/fairness-ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Submission instructions 

All code that you write should be in this notebook.
Submit:

* This notebook with your code added. Make sure to add enough documentation.
* A short report, max 2 pages including any figures and/or tables (it is likely that you won't need the full 2 pages). Use [this template](https://www.overleaf.com/read/mvskntycrckw). 

For questions, make use of the "Lab" session (see schedule).
Questions can also be posted to the MS teams channel called "Lab". 


# Installing AIF360

In this assignment, we're going to use the AIF360 library.
For documentation, take a look at:

    * https://aif360.mybluemix.net/
    * https://aif360.readthedocs.io/en/latest/ (API documentation)
    * https://github.com/Trusted-AI/AIF360 Installation instructions

We recommend using a dedicated Python environment for this assignment, for example
by using Conda (https://docs.conda.io/en/latest/).
You could also use Google Colab (https://colab.research.google.com/).

When installing AIF360, you only need to install the stable, basic version (e.g., pip install aif360)
You don't need to install the additional optional dependencies.

The library itself provides some examples in the GitHub repository, see:
https://github.com/Trusted-AI/AIF360/tree/master/examples.

**Notes**
* The lines below starting with ! can be used in Google Colab. You can also comment them out and manually install these libraries in your console
* The first time you're running the import statements, you may get a warning "No module named tensorflow".
  This can be ignored--we don't need it for this assignment. Just run the code block again, and it should disappear

In [ ]:
!pip install aif360
!pip install fairlearn

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_compas
from aif360.metrics import BinaryLabelDatasetMetric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Exploring the data

**COMPAS dataset**

In this assignment we're going to use the COMPAS dataset.

If you haven't done so already, take a look at this article: https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing.
For background on the dataset, see https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm.

**Reading in the COMPAS dataset**

The AIF360 library has already built in code to read in this dataset.
However, you'll first need to manually download the COMPAS dataset 
and put it into a specified directory. 
See: https://github.com/Trusted-AI/AIF360/blob/master/aif360/data/raw/compas/README.md.
If you try to load in the dataset for the first time, the library will give you instructions on the steps to download the data.

The protected attributes in this dataset are 'sex' and 'race'. 
For this assignment, we'll only focus on race.

The label codes recidivism, which they defined as a new arrest within 2 years. 
Note that in this dataset, the label is coded with 1 being the favorable label.

In [ ]:
!wget -c https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv
!mv compas-scores-two-years.csv /usr/local/lib/python3.10/dist-packages/aif360/data/raw/compas

# # For Google Colab, the DATA_PATH is /usr/local/lib/python3.7/dist-packages/aif360/data/raw/compas
# For Kaggle Notebooks, the DATA_PATH is /opt/conda/lib/python3.7/site-packages/aif360/data/raw/compas

from aif360.datasets import CompasDataset
compas_data = load_preproc_data_compas(protected_attributes=['race'])

--2023-05-02 12:10:35--  https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



Now let's take a look at the data:

In [ ]:
compas_data

               instance weights features                                       \
                                         protected attribute                    
                                     sex                race age_cat=25 to 45   
instance names                                                                  
3                           1.0      0.0                 0.0              1.0   
4                           1.0      0.0                 0.0              0.0   
8                           1.0      0.0                 1.0              1.0   
10                          1.0      1.0                 1.0              1.0   
14                          1.0      0.0                 1.0              1.0   
...                         ...      ...                 ...              ...   
10994                       1.0      0.0                 0.0              1.0   
10995                       1.0      0.0                 0.0              0.0   
10996                       

**Creating a train and test split**

We'll create a train (80%) and test split (20%). 

Note: *Usually when carrying out machine learning experiments,
we also need a dev set for developing and selecting our models (incl. tuning of hyper-parameters).
However, in this assignment, the goal is not to optimize 
the performance of models so we'll only use a train and test split.*

Note: *due to random division of train/test sets, the actual output in your runs may slightly differ with statistics showing in the rest of this notebook.*

In [ ]:
train_data, test_data = compas_data.split([0.8], shuffle=True)

We would also like to know what outcome is favorable in our dataset (because no Recidivism is preferable, it is ```two_year_recid=0``` in our case).

In [ ]:
fav_idx = int(train_data.favorable_label)
print(fav_idx)

0


In this assignment, we'll focus on protected attribute: race.
This is coded as a binary variable with "Caucasian" coded as 1 and "African-American" coded as 0.

In [ ]:
priv_group   = [{'race': 1}]  # Caucasian
unpriv_group = [{'race': 0}]  # African-American

Now let's look at some statistics:

In [ ]:
print("Training set shape: %s, %s" % train_data.features.shape)
print("Favorable (not recid) and unfavorable (recid) labels: %s; %s" % (train_data.favorable_label, train_data.unfavorable_label))
print("Protected attribute names: %s" % train_data.protected_attribute_names)
# labels of privileged (1) and unprovileged groups (0)
print("Privileged (Caucasian) and unprivileged (African-American) protected attribute values: %s, %s" % (train_data.privileged_protected_attributes, 
      train_data.unprivileged_protected_attributes))
print("Feature names: %s" % train_data.feature_names)

Training set shape: 4222, 10
Favorable (not recid) and unfavorable (recid) labels: 0.0; 1.0
Protected attribute names: ['race']
Privileged (Caucasian) and unprivileged (African-American) protected attribute values: [array([1.])], [array([0.])]
Feature names: ['sex', 'race', 'age_cat=25 to 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'priors_count=0', 'priors_count=1 to 3', 'priors_count=More than 3', 'c_charge_degree=F', 'c_charge_degree=M']


Now, let's take a look at the test data and compute the following difference:

$$𝑃(𝑌=favorable|𝐷=unprivileged)−𝑃(𝑌=favorable|𝐷=privileged)$$


In [ ]:
metric_test_data = BinaryLabelDatasetMetric(test_data, 
                             unprivileged_groups = unpriv_group,
                             privileged_groups   = priv_group)
print("Mean difference (statistical parity difference) = %f" % 
      metric_test_data.statistical_parity_difference())


Mean difference (statistical parity difference) = -0.142490


To be clear, because we're looking at the original label distribution this is the base rate difference between the two groups

In [ ]:
metric_test_data.base_rate(False)  # Base rate of the unprivileged group

0.4517133956386293

In [ ]:
metric_test_data.base_rate(True)   # Base rate of the privileged group

0.5942028985507246

To explore the data, it can also help to convert it to a dataframe.
Note that here the favorable label is no recidivism (two_year_recid=0), 
while the mean values are calculated according to two_year_recid=1.
Therefore, we get the results as (1 - base rate).

In [ ]:
test_data.convert_to_dataframe()[0].groupby(['race'])['two_year_recid'].describe()

,count,mean,std,min,25%,50%,75%,max
race,,,,,,,,
0.0,642.0,0.548287,0.498051,0.0,0.0,1.0,1.0,1.0
1.0,414.0,0.405797,0.491640,0.0,0.0,0.0,1.0,1.0


**Report**

Report basic statistics in your report, such as the size of the training and test set.

Now let's explore the *training* data further.
In your report include a short analysis of the training data. Look at the base rates of the outcome variable (two year recidivism) for the combination of both race and sex categories. What do you see?

# Classifiers

**Training classifiers**

Now, train the following classifiers:

1. A logistic regression classifier making use of all features 
2. A logistic regression classifier without the race feature
3. A classifier after reweighting instances in the training set https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.Reweighing.html.
    * Report the weights that are used for reweighing and a short interpretation/discussion.
    * Hint: Think about when you should reweight the data: during initialization or during training (i.e. fit)? Read the documentation of the Logistic regression model in Scikit-learn carefully (if you use it). 
4. A classifier after post-processing 
https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.postprocessing.EqOddsPostprocessing.html#aif360.algorithms.postprocessing.EqOddsPostprocessing 

For training the classifier we recommend using scikit-learn (https://scikit-learn.org/stable/).
AIF360 contains a sklearn wrapper, however that one is in development and not complete.
We recommend using the base AIF360 library, and not their sklearn wrapper.

**Report**

For each of these classifiers, report the following:
* Overall precision, recall, F1 and accuracy.
* The statistical parity difference. Does this classifier satisfy statistical parity? How does this difference compare to the original dataset?
* Difference of true positive rates between the two groups. Does the classifier satisfy the equal opportunity criterion? 



In [ ]:
# FIRST CLASSIFIER: A logistic regression classifier making use of all features
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
model = LogisticRegression(penalty='l2', fit_intercept=True, solver='saga', multi_class='auto', random_state=True, verbose=1)

# Training Model 1
h_model_1 = model.fit(train_data.features, np.reshape(train_data.labels, (-1,)))

# Predicted Labels
y_pred = model.predict(test_data.features)

# True Labels
y_true = np.reshape(test_data.labels, (-1, ))

#Classification Report 
print(classification_report(y_pred, y_true))

convergence after 23 epochs took 0 seconds
              precision    recall  f1-score   support

         0.0       0.71      0.66      0.68       580
         1.0       0.62      0.67      0.64       476

    accuracy                           0.66      1056
   macro avg       0.66      0.66      0.66      1056
weighted avg       0.67      0.66      0.66      1056



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [ ]:
# SECOND CLASSIFIER: A logistic regression classifier without the race feature

# Copy the original dataset (features_to_drop inside CompasDataset did not work)
compas_without_race = (compas_data.convert_to_dataframe()[0]).copy()
# Drop race
compas_without_race.drop('race', axis=1, inplace = True)

train_set, test_set = train_test_split(compas_without_race, test_size=0.2, shuffle=True)

y_train = np.reshape(train_set['two_year_recid'].values, (-1, ))
X_train = train_set.drop('two_year_recid', axis=1)

y_test = np.reshape(test_set['two_year_recid'].values, (-1, ))
X_test = test_set.drop('two_year_recid', axis=1)

# Since the model is the same we omit the (re)definition of the model
h_model_2 = model.fit(X_train, y_train)

# Predicted Labels
y_pred = model.predict(X_test)

# True Labels
y_true = np.reshape(y_test, (-1, ))

#Classification Report 
print(classification_report(y_pred, y_true))

convergence after 25 epochs took 0 seconds
              precision    recall  f1-score   support

         0.0       0.74      0.68      0.71       631
         1.0       0.58      0.65      0.61       425

    accuracy                           0.67      1056
   macro avg       0.66      0.67      0.66      1056
weighted avg       0.68      0.67      0.67      1056



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


In [ ]:
# THIRD CLASSIFIER: A classifier after reweighting instances in the training set
from aif360.algorithms.preprocessing import Reweighing

reweigh = Reweighing()

weights = reweigh.fit(compas_data)

TypeError: ignored

In [ ]:
# FOURTH CLASSIFIER: A classifier after post-processing

# Discussion

**Report**
* Shortly discuss your results. For example, how do the different classifiers compare against each other? 
* Also include a short ethical discussion (1 or 2 paragraphs) reflecting on these two aspects: 1) The use of a ML system to try to predict recidivism; 2) The public release of a dataset like this.
